In [3]:
import os
import voila
import ipywidgets
# import import_ipynb
import fitz
import functools
from Whoosh_Searcher import WhooshHandler
from Whoosh_Searcher import Searcher
from ipywidgets import Button, Layout
from IPython.display import IFrame


"""
    Program responsible for creating the user interface
    ------------------------------------------------------------------------------------------
    Methods:
    mainMenu()
    showHelp(event, outHelp, text)
    defineindex(event, operation)
    createNewIndex(event, msg = "")
    verifyNewIndex(event, indexName)
    confirmNewIndex(event, indexName)
    createConfigureSearch(event, indexName)
    searchKeyword(event, indexName, S, keywords, docType, sortingType, fromDate)
    showPdf(path)
    showImage(event, outImage, image)
    deleteEntry(event, indexName, S, path)
    indexError(indexName,message,operation)
"""

titleLabel = ""
out = ipywidgets.Output(layout={'border': '2px solid black', 'display': 'flex', 'justify_content': 'center'})
outResult = ipywidgets.Output(layout={'border': '2px solid black', 'display': 'flex', 
                                      'justify_content': 'center'})


menuBoxLayout = Layout(left='14.5%')
configureButtonLayout = Layout(left='25%')


def mainMenu():
    """
        Main menu creation.
    
        In this function a menu with two buttons is created, one to open the search configuration menu and 
        another to open the search menu.

    """
    titleIndex = ipywidgets.HTML(value = "<h1 style='font-size:15;text-align:center;'>Welcome to python exercise search engine!</h1>")
    
    configureBtn = ipywidgets.Button(description="Configure Search Space", layout=Layout(width='60%'))
    helpConfBtn = ipywidgets.Button(description="?", layout=Layout(width='10%'))
    confBox = ipywidgets.HBox([configureBtn, helpConfBtn], layout = menuBoxLayout)     
    
    searchBtn = ipywidgets.Button(description="Search", layout=Layout(width='60%'))
    helpSearchBtn = ipywidgets.Button(description="?", layout=Layout(width='10%'))
    searchBox = ipywidgets.HBox([searchBtn, helpSearchBtn], layout = menuBoxLayout)
    
    confHelpText = ("The configure search space menu allows you to configure which directories the program" 
        +" will search for exercises to be used in your searches.")
    confHelpTextBox = ipywidgets.HTML(value = "<p style='border:1px solid lightgray;max-width:510px;padding-left:10px'>"
                                          +confHelpText+"</p>")
    outconfHelp = ipywidgets.Output(layout={'border': '2px'})
    
    
    searchHelpText = ("The search menu allows you to do a personalized search using phrases or words in the directories"
                      +" previously added to the search space.")
    searchHelpTextBox = ipywidgets.HTML(value = "<p style='border:1px solid lightgray;max-width:510px;padding-left:10px'>"
                                          +searchHelpText+"</p>")
    outsearchHelp = ipywidgets.Output(layout={'border': '2px'}) 
    
    helpConfBtn.on_click(functools.partial(showHelp, outHelp = outconfHelp, text = confHelpTextBox))
    helpSearchBtn.on_click(functools.partial(showHelp, outHelp = outsearchHelp, text = searchHelpTextBox))
    
    buttons = [confBox,searchBox]
    
    configureBtn.on_click(functools.partial(defineindex, operation = 'configure'))
    searchBtn.on_click(functools.partial(defineindex, operation = 'search'))
    
    with out:
        display(titleIndex, confBox, searchBox, outconfHelp, outsearchHelp)
    return

def showHelp(event, outHelp, text):
    """
        Help text management.
    
        This function controls the visibility of the text box containing the help text. If the visibility of the 
        received box is visible, the visibility is changed to hidden, otherwise it is changed to visible and the 
        help text is added to the box.
        --------------------------------------------------------------------------------------------------------
        Parameters:
        
        event
        
        outHelp: ipywidgets.Output
            Output box containing help text
        text: ipywidgets.HTML
            Help text box
    """
    
    if(outHelp.layout.visibility == 'visible'):
        outHelp.layout.visibility = 'hidden'
        outHelp.clear_output()
    else:
        outHelp.layout.visibility = 'visible'
        with outHelp: 
            display(text)

def defineindex(event, operation):
    """
        Defining the index.
    
        This function creates the index definition menu. In this menu the user can select one of the previously 
        created indexes present in a Dropdown box. If the user is in the search configuration menu in the 
        dropdown box, he will have an option to create a new index. This menu also contains a help button
        associated with the index selection dropdown box.
        --------------------------------------------------------------------------------------------------------
        Parameters:
        
        event
        
        operation: string{'configure', 'search'}
            Sets the type of menu selected in the main menu
    """
    global titleLabel
    
    title = ""
    
    if(operation == 'configure'):
        title = "Configure Search Menu"
        helpText = ("The index file is the file where the search space directories are defined, select"
                    +" one of the previously created indexes, or select the create new index option to"
                    +" create a new index file.")
        
    elif(operation == 'search'):
        title = "Search Menu"
        helpText = ("The index file is the file where the search space directories are defined, select"
                    +" one of the previously created indexes")
    
    titleLabel = ipywidgets.HTML(value = "<h1 style='font-size:15;text-align:center;width:525px'>"+title+"</h1>")
    
    indexListFile = False
    
    if os.path.exists("IndexFiles.txt"):
        indexListFile = open("IndexFiles.txt","r").readlines()
    
    if (indexListFile):
        titleIndex = ipywidgets.HTML(value = "<b><font-weight='bold'>Choose the index folder name:</b>"
                                     , layout = menuBoxLayout)
        
        helpText += " You have already created the following index files: "
        for file in indexListFile:
            helpText += file.replace('\n', ', ')
        helpText += "."
        
        enterBtn = ipywidgets.Button(description="Enter", layout=Layout(left='33%'))
        
        if(operation == 'configure'):
            indexListFile.append("Create new index")
            
        indexName = ipywidgets.Dropdown(options=indexListFile, value=indexListFile[0], disabled=False
                                        ,layout=Layout(width='60%'))
        helpBtn = ipywidgets.Button(description="?", layout=Layout(width='10%'))
        indexBox = ipywidgets.HBox([indexName, helpBtn], layout = menuBoxLayout)
        
        enterBtn = ipywidgets.Button(description="Enter", layout=Layout(left='33%'))
        
        if(operation == 'configure'):
            enterBtn.on_click(functools.partial(createConfigureSearch, indexName = indexName))
        elif(operation == 'search'):
            enterBtn.on_click(functools.partial(createSearch, indexName = indexName))
        
        
        outHelp = ipywidgets.Output(layout={'border': '2px'})
        helpTextBox = ipywidgets.HTML(value = "<p style='border:1px solid lightgray;max-width:510px;padding-left:10px'>"
                                              +helpText+"</p>")
        helpBtn.on_click(functools.partial(showHelp, outHelp = outHelp, text = helpTextBox))
        

        
        outResult.clear_output()
        with outResult:
            display(titleLabel, titleIndex, indexBox, enterBtn, outHelp)
        
        
    else:       
        createNewIndex(event, "There is no previously created index file, please create your first index file"
                       +" before accessing the search and configure search space menus")
    return

# Index Creation ---------------------------------------------------------------------

def createNewIndex(event, msg = ""):
    """
        Creates a new index.
    
        This function creates the create new index menu. In this menu the user can enter the name of the new 
        index to be created. This menu also contains a help button associated with the index text box and a
        button to cancel this operation.
        --------------------------------------------------------------------------------------------------------
        Parameters:
        
        event
        
        msg: string 
            Message to be displayed in the menu
            Default value = ''
            
    """
    global titleLabel
    
    message = ipywidgets.HTML(value = "<p style='max-width:370px; font-weight: bold;'>"
                              +msg+"</p>", layout = menuBoxLayout)
    titleIndex = ipywidgets.HTML(value = "<b><font-weight='bold'>Insert the new index folder name:</b>"
                                , layout = menuBoxLayout)

    indexName = ipywidgets.Text(placeholder = 'index', layout=Layout(width='60%'))
    helpBtn = ipywidgets.Button(description="?", layout=Layout(width='10%'))
    indexBox = ipywidgets.HBox([indexName, helpBtn], layout = menuBoxLayout) 
    
    helpTextBox = ipywidgets.HTML(value = "<p style='border:1px solid lightgray;max-width:510px;padding-left:10px'>"
                                              +"The index file is the file where the search space directories are" 
                                              +" defined.</p>")
    outHelp = ipywidgets.Output(layout={'border': '2px'})
    helpBtn.on_click(functools.partial(showHelp, outHelp = outHelp, text = helpTextBox))
    
    enterButton = ipywidgets.Button(description="Enter", layout=Layout(width='55%'))
    enterButton.on_click(functools.partial(verifyNewIndex, indexName = indexName))
    cancelButton = ipywidgets.Button(description="Cancel", layout=Layout(width='15%'))
    cancelButton.on_click(functools.partial(defineindex, operation="configure" ))
    btnBox = ipywidgets.HBox([enterButton, cancelButton], layout = menuBoxLayout) 

    outResult.clear_output()
    with outResult:
        display(titleLabel, message, titleIndex, indexBox, btnBox, outHelp)
        
def verifyNewIndex(event, indexName):
    """
        Verify the new index.
    
        This function checks whether the index name is valid or repeated. Only if it is valid and unique does it
        proceed to create the index. If the index name already exists, a warning message is displayed to the 
        user indicating that the index already exists with the option to cancel or continue with the creation.
        --------------------------------------------------------------------------------------------------------
        Parameters:
        
        event
        
        indexName: string 
            Name of the new index      
    """
    global titleLabel
    
    if(indexName.value == ''):
        
        titleIndex = ipywidgets.HTML(value = "<b><font-weight='bold'>Index folder name cant be empty!"
                                     +" Please specify the the index file name.</b>",layout = menuBoxLayout)
        
        goBackBtn = ipywidgets.Button(description="Go back", layout=Layout(left='14.5%', width='72%'))
        goBackBtn.on_click(functools.partial(defineindex, operation = "configure"))
        
        outResult.clear_output()
        with outResult:
            display(titleLabel, titleIndex, goBackBtn)
    else:
        indexListFile = False
        if os.path.exists("IndexFiles.txt"):
            indexListFile = open("IndexFiles.txt","r").readlines()
            indexListFile = [index.replace('\n', '') for index in indexListFile]
        
        if(indexListFile and (indexName.value in indexListFile)):
            titleIndex = ipywidgets.HTML(value = "<p style='text-align:center; max-width:370px; font-weight: bold;'>"
                                         +"There is already an index file with the inserted name , do you wish to"
                                         +" overwrite it ?<p>",layout = menuBoxLayout)
            subTitleIndex = ipywidgets.HTML(value = "<p style='text-align:center; max-width:370px;'>(Attention if you write over the"
                                            +" index file already created, all its content will be lost)"
                                            +"</p>",layout = menuBoxLayout)
            
            yesButton = ipywidgets.Button(description="Yes", layout=Layout(width='35%'))
            yesButton.on_click(functools.partial(confirmNewIndex, indexName = indexName))
            noButton = ipywidgets.Button(description="No", layout=Layout(width='35%'))
            noButton.on_click(createNewIndex)
            btnBox = ipywidgets.HBox([yesButton, noButton], layout = menuBoxLayout) 
        
            outResult.clear_output()
            with outResult:
                display(titleLabel, titleIndex, subTitleIndex, btnBox)
        else:
            confirmNewIndex(event, indexName)

def confirmNewIndex(event, indexName):
    """
        Confirmation of the creation of the new index.
    
        This function calls the method responsible for creating the new index in the class WhooshHandler and 
        presents the result of the creation to the user (If it was successful or unsuccessful) with a button to 
        return to the search configuration menu. 
        --------------------------------------------------------------------------------------------------------
        Parameters:
        
        event
        
        indexName: string 
            Name of the new index      
    """
    global titleLabel
    
    titleIndex = ipywidgets.HTML(value = "<b><font-weight='bold'>Result of the creation of the new index: </b>"
                                 ,layout = menuBoxLayout)
    WH = WhooshHandler(indexName.value+"")
    result = WH.createIndex()
    if (result == None or result == False):
        result = '-Unfortunately it was not possible to create the schema!'
    else:
        result = '-The schema was created successfully!'
    
    resultLabel = ipywidgets.HTML(value = "<b>"+result+"</b>", layout = menuBoxLayout)
    
    returnBtn = ipywidgets.Button(description="Return to index selection", 
                                    layout=Layout(left='14.5%', width='72%'))
    returnBtn.on_click(functools.partial(defineindex, operation = 'configure'))
    
    outResult.clear_output()
    with outResult:
        display(titleLabel, titleIndex, resultLabel, returnBtn)
    
    return
    
            
# Index Configuration ---------------------------------------------------------------------     
def createConfigureSearch(event, indexName):
    """
        Creating the search configuration menu.
    
        This function creates a menu composed of three buttons that perform actions on the selected index. The 
        buttons are, a button for inserting new exercises, a button for defining homework and a button for 
        deleting the index. This menu also contains a help button associated with each of the buttons mentioned
        and a button to cancel this operation.
        --------------------------------------------------------------------------------------------------------
        Parameters:
        
        event
        
        indexName: string 
            Name of the selected index      
    """
    global titleLabel
    if(not isinstance(indexName,str)):
        indexName = (indexName.value).replace('\n', '')
    
    if(indexName == 'Create new index' ):
        createNewIndex(event)
        return
    
    indexListFile = False
    
    if os.path.exists("IndexFiles.txt"):
        indexListFile = open("IndexFiles.txt","r").readlines()
    
    if(indexListFile and ((indexName+"\n") in indexListFile)):
        WH = WhooshHandler(indexName)
        outFolderHelp = ipywidgets.Output(layout={'border': '2px'})
        outHomeWorkHelp = ipywidgets.Output(layout={'border': '2px'})
        outDeleteHelp = ipywidgets.Output(layout={'border': '2px'})
    
        titleIndex = ipywidgets.HTML(value = "<b><font-weight='bold'>Index Name: "+indexName+"</b>",
                                     layout = menuBoxLayout)

        
        insertFolderBtn = ipywidgets.Button(description="Insert Folder of exercises", 
                                            layout=Layout(width='60%'))
        insertFolderBtn.on_click(functools.partial(insertExercises, WH = WH, indexName = indexName))
        helpFolderBtn = ipywidgets.Button(description="?", layout=Layout(width='10%'))  
        insertFolderBox = ipywidgets.HBox([insertFolderBtn, helpFolderBtn], layout = menuBoxLayout)
        
        helpFolderText = ("In the insert folder of exercises menu you can add a new directory to the index "+
                          " ("+indexName+") that contains one or more exercise files.")
        helpFolderTextBox = ipywidgets.HTML(value = "<p style='border:1px solid lightgray;max-width:510px;padding-left:10px'>"
                                          +helpFolderText+"</p>")
        
        helpFolderBtn.on_click(functools.partial(showHelp, outHelp = outFolderHelp, text = helpFolderTextBox))
        
        
        configureHomeWorkBtn = ipywidgets.Button(description="Configure Home Work (Not implemented)",
                                                 layout=Layout(width='60%'))
        helpHomeWorkBtn = ipywidgets.Button(description="?", layout=Layout(width='10%'))  
        insertHomeWorkBox = ipywidgets.HBox([configureHomeWorkBtn, helpHomeWorkBtn], layout = menuBoxLayout)
        
        helpHomeWorText = ("In the configure homework menu ...")
        helpHomeWorTextBox = ipywidgets.HTML(value = "<p style='border:1px solid lightgray;max-width:510px;padding-left:10px'>"
                                          +helpHomeWorText+"</p>")
        
        helpHomeWorkBtn.on_click(functools.partial(showHelp, outHelp = outHomeWorkHelp, text = helpHomeWorTextBox))
        
        
        deleteBtn = ipywidgets.Button(description="Delete this index",layout=Layout(width='60%'))
        deleteBtn.on_click(functools.partial(deleteIndex, WH = WH, indexName = indexName))
        helpDelete = ipywidgets.Button(description="?", layout=Layout(width='10%'))
        deleteBox = ipywidgets.HBox([deleteBtn, helpDelete], layout = menuBoxLayout)
        
        helpDeleteText = ("This action deletes the chosen index file. Warning! proceed with caution this" +
                          " action is irreversible")
        helpDeleteTextBox = ipywidgets.HTML(value = "<p style='border:1px solid lightgray;max-width:510px;padding-left:10px'>"
                                          +helpDeleteText+"</p>")
        helpDelete.on_click(functools.partial(showHelp, outHelp = outDeleteHelp, text = helpDeleteTextBox))
        
        cancelButton = ipywidgets.Button(description="Cancel", layout=Layout(left='14.5%', width='72%'))
        cancelButton.on_click(functools.partial(defineindex, operation="configure" ))
        
        outResult.clear_output()
        with outResult:
            display(titleLabel, titleIndex, insertFolderBox, insertHomeWorkBox, deleteBox, 
                    outFolderHelp, outHomeWorkHelp, outDeleteHelp, cancelButton)
    else:
        titleIndex = ipywidgets.HTML(value = "<b><font-weight='bold'>Error loading index!</b>",
                                     layout = menuBoxLayout)
        subTitleIndex = ipywidgets.HTML(value = "<b><font-weight='bold'>-You need to"
                             +" create the index before you can configure it</b>",layout = menuBoxLayout)
        returnBtn = ipywidgets.Button(description="Return to index selection", 
                                layout=Layout(left='14.5%', width='72%'))
        returnBtn.on_click(functools.partial(defineindex, operation = 'configure'))
        outResult.clear_output()
        with outResult:
            display(titleLabel, titleIndex, subTitleIndex, returnBtn)
    
    return

# Insert Exercises --------------------------------------------------------------------- 
def insertExercises(event, WH, indexName):
    """
        Insert new exercises in the index.
    
        This function allows the user to add exercises to the selected index. For this, the user needs to enter
        the directory where the exercises are located in the text box presented. This menu also contains a help
        button associated with the exercise path text box and a button to cancel this operation.
        --------------------------------------------------------------------------------------------------------
        Parameters:
        
        event
        
        WH: WhooshHandler
            Instance of the created WhoosHandler class, associated with the selected index
        indexName: string 
            Name of the selected index      
    """
    global titleLabel
    
    outHelp = ipywidgets.Output(layout={'border': '2px'})
    
    titleIndex = ipywidgets.HTML(value = "<b><font-weight='bold'>Index Name: "+indexName+"</b>",
                                     layout = menuBoxLayout)
    folderLabel = ipywidgets.HTML(value = "<b>Enter the exercises folder directory path: </b>",
                                     layout = menuBoxLayout)
   
    helpTextBox = ipywidgets.HTML(value = "<p style='border:1px solid lightgray;max-width:510px;padding-left:10px'>"
                                    "Here you can enter the path to the folder where the search for"
                                    +" exercises to be added to the index will be carried out."
                                    +" The program will search for exercises that are in folders with the "
                                    +"following format: "
                                    +"question_x \ version_x \ true_or_false_question.y (x represents a "
                                    +"number and y the exercise format). Each question_x folder can contain"
                                    +" several versions and each version can contain exercises in pdf or tex"
                                    +" format. The more specific and close to the question_x folders the"
                                    +" provided path is, the faster the addition of exercises to the index"
                                    +" will be.</p>")

    pathBox = ipywidgets.Text(placeholder = 'C:\\\ ...', layout = Layout(width='60%'))
    helpBtn = ipywidgets.Button(description="?", layout=Layout(width='10%'))  
    helpBtn.on_click(functools.partial(showHelp, outHelp = outHelp, text = helpTextBox))
    insertExerciseBox = ipywidgets.HBox([pathBox, helpBtn], layout = menuBoxLayout)
    
    insertBtn = ipywidgets.Button(description="Insert", layout=Layout(width='55%'))
    insertBtn.on_click(functools.partial(insertExercisesResult, WH = WH, indexName = indexName, path = pathBox))
    goBackBtn = ipywidgets.Button(description="Cancel", layout=Layout(width='15%'))
    goBackBtn.on_click(functools.partial(createConfigureSearch, indexName = indexName))
    btnBox = ipywidgets.HBox([insertBtn, goBackBtn], layout = menuBoxLayout)
    
    outResult.clear_output()
    with outResult:
        display(titleLabel, titleIndex, folderLabel, insertExerciseBox, btnBox,outHelp)
        
    return    
    

def insertExercisesResult(event, WH, indexName, path):
    """
        Presentation of the result of adding new exercises.
    
        This function calls the method responsible for adding new exercises to the selected index and presents 
        the result of the creation to the user (If it was successful or unsuccessful) with a button to return to
        the search configuration menu. 
        --------------------------------------------------------------------------------------------------------
        Parameters:
        
        event
        
        WH: WhooshHandler
            Instance of the created WhoosHandler class, associated with the selected index
        indexName: string 
            Name of the selected index
        path: string
            Path to the directory where the exercises to be added are
    """
    global titleLabel
    
    result = WH.writeDoc(path.value)
    if(result[0] == False):
        indexError(indexName,result[1],'configure')
        return
    
    titleIndex = ipywidgets.HTML(value = "<b><font-weight='bold'>Index Name: "+indexName+"</b>",
                                     layout = menuBoxLayout)
    resultLabel = ipywidgets.HTML(value = "<p style='max-width:370px;'>"+result[1]+"</p>"
                                  ,layout = menuBoxLayout)
    
    returnBtn = ipywidgets.Button(description="Return to configure search menu", 
                                    layout=Layout(left='14.5%', width='72%'))
    returnBtn.on_click(functools.partial(createConfigureSearch, indexName = indexName))
    
    outResult.clear_output()
    with outResult:
        display(titleLabel, titleIndex, resultLabel, returnBtn)
    
    return

# Insert HomeWork --------------------------------------------------------------------- 
def defineHomeWork(event, WH, indexName):
    global titleLabel
    titleIndex = ipywidgets.Label('Index Name: ' + indexName.value)
    
    
    pathLabel = ipywidgets.Label('Insert the paths for the exercises that compose an homework separating each path with a comma (,)')
    pathBox = ipywidgets.Text(placeholder = 'C:\\\ ...\\\question1, C:\\\ ...\\\question2, ...')
    
    insertBtn = ipywidgets.Button(description="Insert")
    insertBtn.on_click(functools.partial(createHomeWorkResult, WH = WH,indexName = indexName, 
                                         homeWorkPathOut = pathBox))
    outResult.clear_output()
    with outResult:
        display(titleLabel, titleIndex, pathLabel, pathBox, insertBtn)
    

def createHomeWorkResult(event, WH, indexName, homeWorkPathOut):
    global titleLabel
    
    titleIndex = ipywidgets.Label('Index Name: ' + indexName.value)
    
    homeworkArray = homeWorkPathOut.value.replace(" ", "").split(",")
    WH.addHomeWork(homeworkArray)
                       
    result = 'Home Work defined successfully!'        
    resultLabel = ipywidgets.Label(result)
    
    returnBtn = ipywidgets.Button(description="Return to configure search menu", 
                                    layout=Layout(width='30%'))
    returnBtn.on_click(functools.partial(createConfigureSearch, indexName = indexName, WH = WH))
    
    outResult.clear_output()
    with outResult:
        display(titleLabel, titleIndex, resultLabel, returnBtn)
# Delete Index -----------------------------------------------------------------------------

def deleteIndex(event, WH, indexName):
    """
        Elimination of the selected index.
    
        This function calls the method responsible for deleting the selected index and presents 
        the result of the creation to the user (If it was successful or unsuccessful) with a button to 
        return to the search configuration menu. 
        --------------------------------------------------------------------------------------------------------
        Parameters:
        
        event
        
        WH: WhooshHandler
            Instance of the created WhoosHandler class, associated with the selected index
        indexName: string 
            Name of the selected index
    """
    global titleLabel
    
    result = WH.deleteIndex(indexName)
    if(result[0] == False):
        indexError(indexName,result[1],'configure')
        return
    
    titleIndex = ipywidgets.HTML(value = "<b><font-weight='bold'>Index Name: "+indexName+"</b>",
                                     layout = menuBoxLayout)
    resultLabel = ipywidgets.HTML(value = "<p style='max-width:370px;'>"+result[1]+"</p>"
                                  ,layout = menuBoxLayout)
    
    returnBtn = ipywidgets.Button(description="Return to configure search menu", 
                                    layout=Layout(left='14.5%', width='72%'))

    returnBtn.on_click(functools.partial(defineindex, operation = 'configure'))
    
    outResult.clear_output()
    with outResult:
        display(titleLabel, titleIndex, resultLabel, returnBtn)
    
    return

# Search Configuration ---------------------------------------------------------------------   
def createSearch(event, indexName, S=''):
    """
        Creating the search menu.
    
        This function creates a menu composed of the various parameters that the user can select in order to 
        customize his search. The parameters are, a text box where the user can insert the search keywords, a 
        dropdown box to select the type of files to be displayed, a dropdown box to select the age of the files 
        to be presented and a dropdown box to select the result sorting type. This menu also contains a button
        to cancel this operation.
        --------------------------------------------------------------------------------------------------------
        Parameters:
        
        event
        
        indexName: string 
            Name of the selected index 
        S: Searcher
            Instance of the created Searcher class, associated with the selected index
    """
    global titleLabel
    
    if(not isinstance(indexName,str)):
        indexName = (indexName.value).replace('\n', '')
    
    if os.path.exists("IndexFiles.txt"):
        indexListFile = open("IndexFiles.txt","r").readlines()
    
    if(indexListFile and ((indexName+'\n') in indexListFile)):
        if(S == ''):
            S = Searcher(indexName)

        titleIndex = ipywidgets.HTML(value = "<b><font-weight='bold'>Index Name: "+indexName+"</b>",
                                     layout = menuBoxLayout)
        keywordsLabel = ipywidgets.HTML(value = "<b><font-weight='bold'>Type the keywords to be used in your"
                                          +" search</b>",layout = menuBoxLayout)
        keywordsBox = ipywidgets.Text(placeholder = 'Array, random, ...', 
                                      layout = Layout(width='70%', left='14.5%'))

        typeLabel = ipywidgets.HTML(value = "<b><font-weight='bold'>Define the type of documents"
                                          +" search</b>",layout = menuBoxLayout)
        dropDownType = ipywidgets.Dropdown(
            options=['all','pdf', 'py', 'tex'],
            value='pdf',
            description='Type:',
            disabled=False,
            layout = Layout(width='70%', left='14.5%')
        )
        
        dateLabel = ipywidgets.HTML(value = "<b><font-weight='bold'>Define the date of documents"
                                          +" search</b>",layout = menuBoxLayout)
        dropDownDate = ipywidgets.Dropdown(
            options=['all time','this year', 'this month', 'this week'],
            value='all time',
            description='Date:',
            disabled=False,
            layout = Layout(width='70%', left='14.5%')
        )
                       
        sortingLabel = ipywidgets.HTML(value = "<b><font-weight='bold'>Define the sorting type"
                                          +" search</b>",layout = menuBoxLayout)
        dropDownsorting = ipywidgets.Dropdown(
            options=['None','By Number of ocurrences', 'By Date',],
            value='None',
            description='Sorting:',
            disabled=False,
            layout = Layout(width='70%', left='14.5%')
        )

        searchBtn = ipywidgets.Button(description="Search keywords", layout=Layout(width='55%'))
        searchBtn.on_click(functools.partial(searchKeyword, indexName = indexName, S = S,
                                             keywords = keywordsBox, docType = dropDownType,
                                             sortingType = dropDownsorting, fromDate = dropDownDate))
        cancelButton = ipywidgets.Button(description="Cancel", layout=Layout(width='15%'))
        cancelButton.on_click(functools.partial(defineindex, operation="search" ))
        btnBox = ipywidgets.HBox([searchBtn, cancelButton], layout = menuBoxLayout) 

        outResult.clear_output()
        with outResult:
            display(titleLabel, titleIndex, keywordsLabel, keywordsBox, 
                    typeLabel, dropDownType, dateLabel, dropDownDate, sortingLabel, dropDownsorting, btnBox)
    
    
    
    else:
        titleIndex = ipywidgets.HTML(value = "<b><font-weight='bold'>Error loading index!</b>",
                                     layout = menuBoxLayout)
        subTitleIndex = ipywidgets.HTML(value = "<b><font-weight='bold'>-You need to"
                             +" create the index before you can configure it</b>",layout = menuBoxLayout)
        returnBtn = ipywidgets.Button(description="Return to index selection", 
                                layout=Layout(left='14.5%', width='72%'))
        returnBtn.on_click(functools.partial(defineindex, operation = 'configure'))
        outResult.clear_output()
        with outResult:
            display(titleLabel, titleIndex, subTitleIndex, returnBtn)

    return

# Search Result ---------------------------------------------------------------------   
def searchKeyword(event, indexName, S, keywords, docType, sortingType, fromDate):
    """
        Display of search results.
    
        This function creates a list with the results of the search carried out with the parameters chosen by 
        the user.in front of each result in the list there is a button that allows the user to delete the
        exercise from the selected index. If the result is a pdf file in addition to the delete button a 
        show pdf button is also displayed, this button when pressed shows an image with the contents of the pdf 
        file.
        --------------------------------------------------------------------------------------------------------
        Parameters:
        
        event
        
        indexName: string 
            Name of the selected index 
        S: Searcher
            Instance of the created Searcher class, associated with the selected index
        keywords: string
            Keywords to be used in the search
        docType: string
            Type of documents to be shown in the search result
        sortingType: string
            Type of sorting applyed in the search result
        fromDate: string
            Age of the documents to be shown in the search result
    """
        
    results = S.parser(keywords.value, docType.value, sortingType.value, fromDate.value)
    if(results[0] == False):
        indexError(indexName,results[1],'search')
        return
    
    titleLabelMod = ipywidgets.HTML(value = "<h1 style='font-size:15;text-align:center;'>"
                                                 "Search Result</h1>")
    
    titleIndex = ipywidgets.HTML(value = "<b><font-weight='bold'>Index Name: "+indexName+"</b>")
    returnBtn = ipywidgets.Button(description="Return to search menu", 
                                    layout=Layout(width='30%'))
    returnBtn.on_click(functools.partial(createSearch, indexName = indexName, S = S))

    
    outResult.clear_output()
    i = 0
    with outResult:
        display(titleLabelMod, titleIndex, returnBtn)
        if (results != None):
            for result in results[1]:

                path = result[0]
                pathLabel = ipywidgets.Label(str(i)+ ' - ' + path)
                i += 1
                
                deleteBtn = ipywidgets.Button(description="Delete", 
                                    layout=Layout(width='10%', align_content='flex-end'))
                
                
                deleteBtn.on_click(functools.partial(deleteEntry, S = S, indexName = indexName,
                                                     path = os.path.dirname(os.path.dirname(path)) ))

                if(result[1] == 'pdf'):
                    image = showPdf(path)

                    outImage = ipywidgets.Output(layout={'border': '1px solid black', 'visibility': 'hidden'})


                    showImageBtn = ipywidgets.Button(description="Open PDF", 
                                    layout=Layout(width='15'))
                    showImageBtn.on_click(functools.partial(showImage, outImage = outImage, image = image))

                    pdfHBox = ipywidgets.HBox([pathLabel, showImageBtn, deleteBtn])               
                    display(pdfHBox, outImage)
                else:
                    display(ipywidgets.HBox([pathLabel, deleteBtn]))

                display(ipywidgets.Label(''))
        else:
            display(ipywidgets.Label('There are no results for this search'))
    return

def showPdf(path):
    """
        Creating the pdf image.
    
        This function creates an image with the contents of the pdf represented by the received path.
        --------------------------------------------------------------------------------------------------------
        Parameters:
        
        event
        
        path: string 
            Path to de pdf file
        --------------------------------------------------------------------------------------------------------
        Returns
        
        image: ipywidgets.Image
            Image of the pdf to be shown
    """
    doc = fitz.open(path)
    page = doc.loadPage(0)
    pix = page.getPixmap()
    png = pix.getPNGData()
    
    image = ipywidgets.Image(
                value=png,
                format='png',
                width=400,
                height='auto'
            )
    
    return image
    
def showImage(event, outImage, image):
    """
        Pdf image management.
    
        This function controls the visibility of the image box containing the contents of a pdf file. If the 
        visibility of the received image box is visible, the visibility is changed to hidden, otherwise it is 
        changed to visible and the image is added to the box.
        --------------------------------------------------------------------------------------------------------
        Parameters:
        
        event
        
        outImage: ipywidgets.Output
            Output box containing the image
        text: ipywidgets.ipywidgets.Image
            Image of the pdf file
    """
    
    if(outImage.layout.visibility == 'visible'):
        outImage.layout.visibility = 'hidden'
        outImage.clear_output()
    else:
        outImage.layout.visibility = 'visible'
        with outImage: 
            display(image)
# Delete Document -------------------------------------------------------------------------

def deleteEntry(event, indexName, S, path):
    """
        Elimination of the selected exercise from the selected index.
    
        This function calls the method responsible for deleting the selected exercise and presents 
        the result of the elimination to the user (If it was successful or unsuccessful) with a button to 
        return to the search menu. .
        --------------------------------------------------------------------------------------------------------
        Parameters:
        
        event
        
        indexName: string 
            Name of the selected index
        S: Searcher
            Instance of the created Searcher class, associated with the selected index
        path: string
            Path of the selected exercise
        
    """
    global titleLabel
    
    result = S.deleteEntry(path)
    if(result[0] == False):
        indexError(indexName,result[1], 'search')
        return
    
    titleIndex = ipywidgets.HTML(value = "<b><font-weight='bold'>Index Name: "+indexName+"</b>",
                                     layout = menuBoxLayout)
    resultLabel = ipywidgets.HTML(value = "<p style='max-width:370px;'>"+result[1]+"</p>"
                                  ,layout = menuBoxLayout)
    
    returnBtn = ipywidgets.Button(description="Return to search menu", 
                                    layout=Layout(left='14.5%', width='72%'))

    returnBtn.on_click(functools.partial(createSearch, indexName = indexName, S = S))
    
    outResult.clear_output()
    with outResult:
        display(titleLabel, titleIndex, resultLabel, returnBtn)
    
    return

# Index Error -----------------------------------------------------------------------------
    
def indexError(indexName,message,operation):
    """
        Show error messages.
    
        In case of an error this function is called in order to construc a menu with This function creates a 
        menu with the description and reason for the error.
        --------------------------------------------------------------------------------------------------------
        Parameters:
        
        event
        
        indexName: string 
            Name of the selected index
        S: Searcher
            Instance of the created Searcher class, associated with the selected index
        path: string
            Path of the selected exercise
        
    """
    global titleLabel
    titleIndex = ipywidgets.HTML(value = "<b><font-weight='bold'>Index Name: "+indexName+"</b>",
                                     layout = menuBoxLayout)

    error = ipywidgets.HTML(value = "<p style='max-width:370px;'>"+message+"</p>"
                                  ,layout = menuBoxLayout)
    
    returnBtn = ipywidgets.Button(description="Return to "+ operation +" menu", 
                                    layout=Layout(left='14.5%', width='72%'))
    returnBtn.on_click(functools.partial(defineindex, operation = operation))

    outResult.clear_output()
    with outResult:
        display(titleLabel, titleIndex, error, returnBtn)
    return

mainMenu()
display(out)
display(outResult)



Output(layout=Layout(border='2px solid black', display='flex', justify_content='center'))

Output(layout=Layout(border='2px solid black', display='flex', justify_content='center'))